This notebook is just for computing the accuracy on the adversarial examples. 

In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
import pickle
import time
from skimage.measure import compare_ssim
import tensorflow as tf
from keras.models import Model,load_model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, Activation
from keras.utils import to_categorical
import keras
from sklearn.metrics import accuracy_score
import random


Using TensorFlow backend.


In [2]:
#Set seeds
random.seed(1)
np.random.seed(1)

In [3]:
# Network 3 
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

CIFAR_IMG= 32

input_shape = (CIFAR_IMG,CIFAR_IMG,1)
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=input_shape))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
 
model.summary()


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


'''
history = model.fit(x=x_train_C,y=y_train_C, epochs=150, batch_size=64, validation_data=[x_test_C,y_test_C])

score= model.evaluate(x_test_C, y_test_C,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save_weights("CNN_CIFAR10_net3.h5")
files.download('CNN_CIFAR10_net3.h5')
'''

model = load_model("../Models/CNN_CIFAR10_net3.h5")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)       

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_sc = X_train / 255.0
X_test_sc = X_test / 255.0
y_cat = to_categorical(y_train)

def grayscale(data, dtype='float32'):
    # luma coding weighted average in video systems
    r, g, b = np.asarray(.3, dtype=dtype), np.asarray(.59, dtype=dtype), np.asarray(.11, dtype=dtype)
    rst = r * data[:, :, :, 0] + g * data[:, :, :, 1] + b * data[:, :, :, 2]
    # add channel dimension
    rst = np.expand_dims(rst, axis=3)
    return rst

X_sc = grayscale(X_sc)
X_test_sc = grayscale(X_test_sc)
X_train = grayscale(X_train)
X_test = grayscale(X_test)

y_pred_test = model.predict(np.expand_dims(X_test_sc.reshape(X_test_sc.shape[0],32,32),axis=3))
y_pred_train = model.predict(np.expand_dims(X_sc.reshape(X_sc.shape[0],32,32),axis=3))


In [67]:
print("Training accuracy: ",round(accuracy_score(y_train,np.argmax(y_pred_train,axis=1)),3))
print("Test accuracy: ",round(accuracy_score(y_test,np.argmax(y_pred_test,axis=1)),3))

Training accuracy:  0.958
Test accuracy:  0.824


In [6]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [7]:
# make subset 
def sample_test_set(percentage,X_test,X_test_scaled,y_test,imsize,channels,num_clusters, num_classes,num_samples):
    
    original = []
    adversarial = []
    original_y = []
    adversarial_y = []
    for sample_class in range(num_classes):
        labels = y_test[(y_test==sample_class).reshape(num_samples)]
        samples_orig = X_test[(y_test==sample_class).reshape(num_samples)]
        samples = X_test_scaled[(y_test==sample_class).reshape(num_samples)]
        samples = samples.reshape((samples.shape[0],imsize*imsize*channels))
        kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit_predict(samples)
        for cluster in range(num_clusters):  
            subsamples = np.where(kmeans==cluster)[0]
            ori, adv = train_test_split(subsamples,random_state=0, test_size = percentage ,shuffle=True)
            original.extend(samples_orig[ori])
            original_y.extend(labels[ori])
            adversarial.extend(samples_orig[adv])
            adversarial_y.extend(labels[adv])
    return np.array(original), np.array(adversarial), np.array(original_y), np.array(adversarial_y)

In [68]:
discard,subsample, discard_y, subsample_y = sample_test_set(0.1,X_test,X_test_sc,y_test,32,1,10,10,10000)

In [69]:
print(f"Subsample : {subsample.shape}")
print(f"Subsample_y: {subsample_y.shape}")


Subsample : (1047, 32, 32, 1)
Subsample_y: (1047, 1)


In [70]:
subsample_sc = subsample / 255.0
y_pred_subsample = model.predict(np.expand_dims(subsample_sc.reshape(subsample_sc.shape[0],32,32),axis=3))
print("Accuracy on subset before: ", round(accuracy_score(subsample_y,np.argmax(y_pred_subsample,axis=1)),3))

Accuracy on subset before:  0.824


In [11]:
original,adversarial, original_y, adversarial_y = sample_test_set(0.1,subsample,subsample_sc,subsample_y,32,1,6,10,1047)

In [71]:
original_sc = original / 255.0
y_pred_original = model.predict(np.expand_dims(original_sc.reshape(original_sc.shape[0],32,32),axis=3))
print("Accuracy on original: ", accuracy_score(original_y,np.argmax(y_pred_original,axis=1)))

adversarial_sc = adversarial / 255.0
y_pred_adversarial = model.predict(np.expand_dims(adversarial_sc.reshape(adversarial_sc.shape[0],32,32),axis=3))
print("Accuracy on adversarial before: ", round(accuracy_score(adversarial_y,np.argmax(y_pred_adversarial,axis=1)),3))

Accuracy on original:  0.824945295404814
Accuracy on adversarial before:  0.82


In [72]:
with open('../subsets/subset_cifar', 'rb') as f:
    original = pickle.load(f)
    adversarial= pickle.load(f)
    original_y = pickle.load(f)
    adversarial_y = pickle.load(f)

print(f"Original shape {original.shape}")
print(f"Adversarial shape {adversarial.shape}")
print(f"Original labels shape {original_y.shape}")
print(f"Adversarial labels shape {adversarial_y.shape}")

Original shape (914, 32, 32, 1)
Adversarial shape (133, 32, 32, 1)
Original labels shape (914, 1)
Adversarial labels shape (133, 1)


PSNR

In [82]:
#https://www.geeksforgeeks.org/python-peak-signal-to-noise-ratio-psnr/
from math import log10, sqrt 
import cv2 

def PSNR(original,evolved):
    psnr=[]
    for i in range(original.shape[0]):
        psnr_original = cv2.PSNR(original[i],original[i])
        psnr_adv_org = cv2.PSNR(original[i],np.array(evolved[i],dtype="float32"))
        psnr.append(round((psnr_adv_org/psnr_original)*100,3))
    return psnr

'''def PSNR(original, compressed): 
    mse = np.mean((original - compressed) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr
    
'''

'def PSNR(original, compressed): \n    mse = np.mean((original - compressed) ** 2) \n    if(mse == 0):  # MSE is zero means no noise is present in the signal . \n                  # Therefore PSNR have no importance. \n        return 100\n    max_pixel = 255.0\n    psnr = 20 * log10(max_pixel / sqrt(mse)) \n    return psnr\n    \n'

MULTICROSSOVER

In [75]:
with open('./results/GA_adv_cifar_multicrossover_seed1', 'rb') as f:
    evolved_examples = pickle.load(f)
    times= pickle.load(f)
    ssim_values = pickle.load(f)
    fitness_of_evolved = pickle.load(f)
    predicted_class = pickle.load(f)
    number_of_rounds = pickle.load(f)

In [76]:
np.array(evolved_examples).shape

(133, 1024)

In [77]:
# computing accuracy on adversarial after 
evolved_examples = np.array(evolved_examples)
evolved_examples=evolved_examples/ 255.0
y_pred_adversarial = model.predict(np.expand_dims(evolved_examples.reshape(evolved_examples.shape[0],32,32),axis=3))
print("Accuracy on adversarial after: ", round(accuracy_score(adversarial_y,np.argmax(y_pred_adversarial,axis=1)),3))

Accuracy on adversarial after:  0.083


In [87]:
subsample = np.concatenate((original,evolved_examples.reshape(evolved_examples.shape[0],32,32,1)))
print(subsample.shape)

subsample_sc = subsample / 255.0
subsample_y = np.concatenate((original_y,adversarial_y))
y_pred_subsample = model.predict(np.expand_dims(subsample_sc.reshape(subsample_sc.shape[0],32,32),axis=3))
print("Accuracy on subsample after: ", round(accuracy_score(subsample_y,np.argmax(y_pred_subsample,axis=1)),3))

(1047, 32, 32, 1)
Accuracy on subsample after:  0.734


In [86]:
psnr = PSNR(adversarial.reshape(adversarial.shape[0],32,32),evolved_examples.reshape(evolved_examples.shape[0],32,32))

print(f"Mean psnr {round(np.mean(psnr),3)}")
print(f"Std psnr {round(np.std(psnr),3)}")
print(f"Min psnr {round(np.min(psnr),3)}")
print(f"Max psnr {round(np.max(psnr),3)}")

Mean psnr 1.6
Std psnr 0.558
Min psnr 0.24
Max psnr 3.331


2-K CROSSOVER

In [90]:
with open('./results/GA_adv_cifar_ssim_Kcrossover_seed1', 'rb') as f:
    evolved_examples = pickle.load(f)
    times= pickle.load(f)
    ssim_values = pickle.load(f)
    fitness_of_evolved = pickle.load(f)
    predicted_class = pickle.load(f)
    number_of_rounds = pickle.load(f)

In [91]:
# computing accuracy on adversarial after 
evolved_examples = np.array(evolved_examples)
evolved_examples=evolved_examples/ 255.0
y_pred_adversarial = model.predict(np.expand_dims(evolved_examples.reshape(evolved_examples.shape[0],32,32),axis=3))
print("Accuracy on adversarial after: ", round(accuracy_score(adversarial_y,np.argmax(y_pred_adversarial,axis=1)),3))

Accuracy on adversarial after:  0.06


In [92]:
subsample = np.concatenate((original,evolved_examples.reshape(evolved_examples.shape[0],32,32,1)))
print(subsample.shape)

subsample_sc = subsample / 255.0
subsample_y = np.concatenate((original_y,adversarial_y))
y_pred_subsample = model.predict(np.expand_dims(subsample_sc.reshape(subsample_sc.shape[0],32,32),axis=3))
print("Accuracy on subsample: ", round(accuracy_score(subsample_y,np.argmax(y_pred_subsample,axis=1)),3))

(1047, 32, 32, 1)
Accuracy on subsample:  0.734


In [93]:
# computing PSNR

psnr = PSNR(adversarial.reshape(adversarial.shape[0],32,32),evolved_examples.reshape(evolved_examples.shape[0],32,32))

print(f"Mean psnr {round(np.mean(psnr),3)}")
print(f"Std psnr {round(np.std(psnr),3)}")
print(f"Min psnr {round(np.min(psnr),3)}")
print(f"Max psnr {round(np.max(psnr),3)}")

Mean psnr 1.6
Std psnr 0.558
Min psnr 0.24
Max psnr 3.331


QUADRANT CROSSOVER 

In [94]:
with open('./results/GA_adv_cifar_quadrant_seed1', 'rb') as f:
    evolved_examples = pickle.load(f)
    times= pickle.load(f)
    ssim_values = pickle.load(f)
    fitness_of_evolved = pickle.load(f)
    predicted_class = pickle.load(f)
    number_of_rounds = pickle.load(f)

In [96]:
# computing accuracy on adversarial after 
evolved_examples = np.array(evolved_examples)
evolved_examples=evolved_examples/ 255.0
y_pred_adversarial = model.predict(np.expand_dims(evolved_examples.reshape(evolved_examples.shape[0],32,32),axis=3))
print("Accuracy on adversarial after: ", round(accuracy_score(adversarial_y,np.argmax(y_pred_adversarial,axis=1)),3))

Accuracy on adversarial after:  0.105


In [97]:
# computing accuracy on subset after
#concatenating original and evolved images
subsample = np.concatenate((original,evolved_examples.reshape(evolved_examples.shape[0],32,32,1)))
print(subsample.shape)

subsample_sc = subsample / 255.0
#concatenating the ground truth labels original + adversarial
subsample_y = np.concatenate((original_y,adversarial_y))
y_pred_subsample = model.predict(np.expand_dims(subsample_sc.reshape(subsample_sc.shape[0],32,32),axis=3))
print("Accuracy on subsample after: ", round(accuracy_score(subsample_y,np.argmax(y_pred_subsample,axis=1)),3))

(1047, 32, 32, 1)
Accuracy on subsample after:  0.734


In [99]:
# computing PSNR
psnr = PSNR(adversarial.reshape(adversarial.shape[0],32,32),evolved_examples.reshape(evolved_examples.shape[0],32,32))

print(f"Mean psnr {round(np.mean(psnr),3)}")
print(f"Std psnr {round(np.std(psnr),3)}")
print(f"Min psnr {round(np.min(psnr),3)}")
print(f"Max psnr {round(np.max(psnr),3)}")


print(f"Mean psnr {np.mean(psnr)}")
print(f"Std psnr {np.std(psnr)}")
print(f"Min psnr {np.min(psnr)}")
print(f"Max psnr {np.max(psnr)}")

Mean psnr 1.591
Std psnr 0.558
Min psnr 0.231
Max psnr 3.322
Mean psnr 1.5907744360902256
Std psnr 0.5582019148053428
Min psnr 0.231
Max psnr 3.322


UNIFORM 

In [100]:
with open('./results/GA_adv_cifar_ssim_uniform_seed1', 'rb') as f:
    evolved_examples = pickle.load(f)
    times= pickle.load(f)
    ssim_values = pickle.load(f)
    fitness_of_evolved = pickle.load(f)
    predicted_class = pickle.load(f)
    number_of_rounds = pickle.load(f)

In [101]:
# computing accuracy on adversarial after 
evolved_examples = np.array(evolved_examples)
evolved_examples=evolved_examples/ 255.0
y_pred_adversarial = model.predict(np.expand_dims(evolved_examples.reshape(evolved_examples.shape[0],32,32),axis=3))
print("Accuracy on adversarial after: ", round(accuracy_score(adversarial_y,np.argmax(y_pred_adversarial,axis=1)),3))

Accuracy on adversarial after:  0.075


In [102]:
# computing accuracy on subset after
#concatenating original and evolved images
subsample = np.concatenate((original,evolved_examples.reshape(evolved_examples.shape[0],32,32,1)))
print(subsample.shape)

subsample_sc = subsample / 255.0
#concatenating the ground truth labels original + adversarial
subsample_y = np.concatenate((original_y,adversarial_y))
y_pred_subsample = model.predict(np.expand_dims(subsample_sc.reshape(subsample_sc.shape[0],32,32),axis=3))
print("Accuracy on subsample after: ", round(accuracy_score(subsample_y,np.argmax(y_pred_subsample,axis=1)),3))

(1047, 32, 32, 1)
Accuracy on subsample after:  0.734


In [103]:
psnr = PSNR(adversarial.reshape(adversarial.shape[0],32,32),evolved_examples.reshape(evolved_examples.shape[0],32,32))

print(f"Mean psnr {round(np.mean(psnr),3)}")
print(f"Std psnr {round(np.std(psnr),3)}")
print(f"Min psnr {round(np.min(psnr),3)}")
print(f"Max psnr {round(np.max(psnr),3)}")

Mean psnr 1.6
Std psnr 0.558
Min psnr 0.24
Max psnr 3.331
